In [ ]:
from stanza.utils.visualization.dependency_visualization import visualize_strings

ar_strings = ['برلين ترفض حصول شركة اميركية على رخصة تصنيع دبابة "ليوبارد" الالمانية', "هل بإمكاني مساعدتك؟", 
              "أراك في مابعد", "لحظة من فضلك"]
# Testing with right to left language
visualize_strings(ar_strings, "ar")

In [4]:
from stanza.utils.visualization.dependency_visualization import visualize_strings

en_strings = ["This is a sentence.", 
              "He is wearing a red shirt",
              "Barack Obama was born in Hawaii. He was elected President of the United States in 2008."]
# Testing with left to right languages
visualize_strings(en_strings, "en")

2022-07-14 16:37:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-07-14 16:37:51 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-07-14 16:37:51 INFO: Use device: gpu
2022-07-14 16:37:51 INFO: Loading: tokenize
2022-07-14 16:37:53 INFO: Loading: pos
2022-07-14 16:37:53 INFO: Loading: lemma
2022-07-14 16:37:53 INFO: Loading: depparse
2022-07-14 16:37:54 INFO: Done loading processors!


In [ ]:
from stanza.utils.visualization.dependency_visualization import visualize_strings

zh_strings = ["中国是一个很有意思的国家。"]
# Testing with right to left language
visualize_strings(zh_strings, "zh")